In [1]:
# first, we import the relevent packages.

# for data manipulation
import pandas as pd
import datetime as dt

# to scrape the web
from bs4 import BeautifulSoup as soupy
import requests
import time
import random

# dealing with consistent ssl error
import ssl

# for maths
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for writing dictionaries and others as files
import json

In [2]:
url = 'https://en.wikipedia.org/wiki/Party_divisions_of_United_States_Congresses'

In [3]:
# temporary ssl fix
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# grab the table
table = pd.read_html(url, attrs={"class": "wikitable"})

In [5]:
# make the data frame
table = pd.DataFrame(table[0])

In [6]:
# get the columns
table.columns = table.iloc[1119]

In [7]:
# make a slice
table = table.iloc[1088:1118, [0,7,8,9,12,13]].copy(deep = True)

In [8]:
# find the democratic advantage
table['Partisanship_Elected'] = table['Democrats'].astype(int) - table['Republicans'].astype(int)

In [9]:
# remove the refrences
table['President'] = table.President.str.replace('2','')
table['President'] = table.President.str.replace('4','')
table['Trifecta'] = table.Trifecta.str.replace('3','')
table['Trifecta'] = table.Trifecta.str.replace('5','')
table['Trifecta'] = table.Trifecta.str.replace('1','')
table['President'] = table.President.str.replace('[[]]','')
table['Trifecta'] = table.Trifecta.str.replace('[[]]','')

# standardize text
table['Trifecta'] = table.Trifecta.str.replace('Yes/No','1')
table['Trifecta'] = table.Trifecta.str.replace('Yes','2')
table['Trifecta'] = table.Trifecta.str.replace('No','0')

# deal with presidents
table['President'] = table.President.str.replace('Dwight D. Eisenhower','1')
table['President'] = table.President.str.replace('John F. Kennedy','1')
table['President'] = table.President.str.replace('Lyndon B. Johnson','1')
table['President'] = table.President.str.replace('Richard Nixon','0')
table['President'] = table.President.str.replace('Gerald Ford','0')
table['President'] = table.President.str.replace('Jimmy Carter','1')
table['President'] = table.President.str.replace('Ronald Reagan','0')
table['President'] = table.President.str.replace('George H. W. Bush','0')
table['President'] = table.President.str.replace('Bill Clinton','1')
table['President'] = table.President.str.replace('George W. Bush','0')
table['President'] = table.President.str.replace('Barack Obama','1')
table['President'] = table.President.str.replace('Donald Trump','0')
table['President_Democrat'] = table['President']

# get rid of alpha charachters, I think this is ultimately faster than using isalpha
table['Congress'] = table.Congress.str.replace('th','')
table['Congress'] = table.Congress.str.replace('st','')
table['Congress'] = table.Congress.str.replace('nd','')
table['Congress'] = table.Congress.str.replace('rd','')

# data type
table['Congress'] = table.Congress.astype(int)
table['Partisanship_Elected'] = table.Partisanship_Elected.astype(int)
table['President_Democrat'] = table.President_Democrat.astype(int)

C:\Users\cmacd\AppData\Local\Temp/ipykernel_8068/3928101674.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  table['President'] = table.President.str.replace('[22]','')
C:\Users\cmacd\AppData\Local\Temp/ipykernel_8068/3928101674.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  table['President'] = table.President.str.replace('[24]','')
C:\Users\cmacd\AppData\Local\Temp/ipykernel_8068/3928101674.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  table['Trifecta'] = table.Trifecta.str.replace('[23]','')
C:\Users\cmacd\AppData\Local\Temp/ipykernel_8068/3928101674.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  table['Trifecta'] = table.Trifecta.str.replace('[25]','')
C:\Users\cmacd\AppData\Local\Temp/ipykernel_8068/3928101674.py:5: FutureWarning: The default value of regex will cha

In [10]:
#the even years in this span were midterm elections
list_of_midterms = []
for i in range(30):
    if i%2 == 0:
        list_of_midterms.append(1)
    else:
        list_of_midterms.append(0)

In [11]:
# add the values to data frame after making a series
series_of_midterms = pd.Series(list_of_midterms)
table['Midterm_Next'] = series_of_midterms.values

In [16]:
# drop some index and columns
table = table.drop(columns = ['Total', 'Democrats', 'Republicans', 'President']).reset_index(drop = True)

In [17]:
# export to data
table.to_csv('data/partisan.csv', encoding='utf-8', index=False)